In [18]:
from ortools.sat.python import cp_model
import pandas as pd
from datetime import datetime, timedelta

start_date = datetime.strptime('2023-07-01', '%Y-%m-%d')

# Create the model.
model = cp_model.CpModel()

## Constants
num_employees = 6
num_days = 365
shift_types = ['Morning', 'Afternoon', 'Night', 'Day off']

# Variables
shifts = {}
total_shifts = []
for j in range(num_employees):
    for i in range(num_days):
        for k in range(len(shift_types)):
            shift_var = model.NewBoolVar('shift_%i%i%i' % (j, i, k))
            shifts[(j, i, k)] = shift_var
            if k != 3:  # Don't count 'Day off' as a shift
                total_shifts.append(shift_var)

# Constraints

# Each shift every day is assigned to exactly one employee,
# or the employee has a day off
for i in range(num_days):
    for k in range(len(shift_types) - 1):  # We don't need to assign 'Day off'
        model.Add(sum(shifts[(j, i, k)] for j in range(num_employees)) == 1)

# Every employee must have 1 shift or a day off every day
for j in range(num_employees):
    for i in range(num_days):
        model.Add(sum(shifts[(j, i, k)] for k in range(len(shift_types))) == 1)

# No employee works two shifts in the same day
for j in range(num_employees):
    for i in range(num_days):
        model.Add(sum(shifts[(j, i, k)] for k in range(len(shift_types))) <= 1)

# Each employee works no more than 5 days in a row
for j in range(num_employees):
    for i in range(num_days - 5):
        model.Add(sum(shifts[(j, i + d, k)] for d in range(6) for k in range(len(shift_types) - 1)) <= 5)

# Each employee works no more than 2 nights in a row
for j in range(num_employees):
    for i in range(num_days - 3):
        model.Add(sum(shifts[(j, i + d, 2)] for d in range(3)) <= 2)

# Implementing the transition constraints
for j in range(num_employees):
    for i in range(num_days - 1):
        # If Morning, next shift can be Morning, Afternoon or Day off
        morning_next = model.NewBoolVar('morning_next_%i%i' % (j, i))
        model.AddBoolOr([shifts[(j, i + 1, 0)], shifts[(j, i + 1, 1)], shifts[(j, i + 1, 3)], morning_next.Not()])
        model.AddImplication(shifts[(j, i, 0)], morning_next)
        # If Afternoon, next shift can be Afternoon, Night or Day off
        afternoon_next = model.NewBoolVar('afternoon_next_%i%i' % (j, i))
        model.AddBoolOr([shifts[(j, i + 1, 1)], shifts[(j, i + 1, 2)], shifts[(j, i + 1, 3)], afternoon_next.Not()])
        model.AddImplication(shifts[(j, i, 1)], afternoon_next)
        # If Night, next shift can be Night or Day off
        night_next = model.NewBoolVar('night_next_%i%i' % (j, i))
        model.AddBoolOr([shifts[(j, i + 1, 2)], shifts[(j, i + 1, 3)], night_next.Not()])
        model.AddImplication(shifts[(j, i, 2)], night_next)


# Creating auxiliary variables
max_shifts = model.NewIntVar(0, num_days, 'max_shifts')
min_shifts = model.NewIntVar(0, num_days, 'min_shifts')

# Constraint to keep track of the max and min shifts
for j in range(num_employees):
    model.Add(max_shifts >= total_shifts_per_worker[j])
    model.Add(min_shifts <= total_shifts_per_worker[j])

# Minimize the difference between the employee who works the most and the one who works the least
model.Minimize(max_shifts - min_shifts)

# Creates the solver and solve.
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the results and Prepare the data
roster_data = []
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Found solution!')
    for i in range(num_days):
        for j in range(num_employees):
            worked = False
            for k in range(len(shift_types)):
                if solver.BooleanValue(shifts[(j, i, k)]):
                    print(f"Employee {j} works on day {i} shift {shift_types[k]}")
                    current_date = (start_date + timedelta(days=i)).strftime('%Y-%m-%d')
                    roster_data.append([current_date, j, shift_types[k]])
                    worked = True
            if not worked: # If no shift was worked, add a day off
                roster_data.append([current_date, j, 'Day off'])
else:
    print('No solution found !')

# Create a pandas DataFrame
roster_df = pd.DataFrame(roster_data, columns=['Date', 'Employee', 'Shift'])

# Pivot DataFrame so that employees are columns
roster_pivot_df = roster_df.pivot(index='Date', columns='Employee', values='Shift')

# Write to Excel
roster_pivot_df.to_excel('roster.xlsx')


RecursionError: maximum recursion depth exceeded while getting the str of an object